# Exercise 2 - Celestial Mechanics
Giacomo Menegatti (2122852)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp 
from scipy.interpolate import interp1d


## Explicit implementation of a Runge-Kutta integrator

In [10]:
def RK(f, y0, t0, tf, h):
  #Creating two arrays
  Y = [y0]
  t = [t0]

  while t_last < tf:
    
    t_last = t[-1]
    y = Y[-1]

    #Runge Kutta coefficients
    k1 = h*f(t,y)
    k2 = h*f(t + 0.5*h, y + 0.5*k1)
    k3 = h*f(t + 0.5*h, y + 0.5*k2)
    k4 = h*f(t + h, y + k3)
        
    y = y + (k1 + 2*k2 + 2*k3 + k4)/6.0
    t_last  = t_last + h

    Y.append(y)
    t.append(t_last)
  # I transpose the array to have the same shape as the scipy one
  # meaning [[y1(t0),...,y1(tf)],....,[yn(t0),...,yn(tf)]]
    
  #Dense output
  Y = interp1d(t, np.array(Y).T) 

  return t, Y



## Adaptive step Runge-Kutta
The code is equal to the one before, just now h is a function of t and y instead of a constant

In [11]:
def RK_Adaptive(f, y0, t0, tf, h):

  #Creating two arrays
  Y = [y0]
  t = [t0]

  while t_last < tf:
    
    t_last = t[-1]
    y = Y[-1]
    h = h(t_last, t_last)

    #Runge Kutta coefficients
    k1 = h*f(t,y)
    k2 = h*f(t + 0.5*h, y + 0.5*k1)
    k3 = h*f(t + 0.5*h, y + 0.5*k2)
    k4 = h*f(t + h, y + k3)
        
    y = y + (k1 + 2*k2 + 2*k3 + k4)/6.0
    t_last  = t_last + h

    Y.append(y)
    t.append(t_last)
  # I transpose the array to have the same shape as the scipy one
  # meaning [[y1(t0),...,y1(tf)],....,[yn(t0),...,yn(tf)]]
  #Dense output
  Y = interp1d(t, np.array(Y).T) 

  return t, Y


# Two-body integration

In [12]:
# ODE function
def two_body_ODE(t,y, GM):
  #Dividing y in positions and velocities
  r1, r2  = y[0:2], y[2:4]  #The first four variables are positions
  v1, v2 = y[4:6], y[6:8]  #The last four are velocities
  #Calculating the accelerations
  r_12 = r2-r1
  r3 = np.linalg.norm(r_12)**3
  #With concatenate I concatenate vectors side by side to get the state vector

  return np.concatenate((v1, v2, GM/r3*r_12, -GM/r3*r_12), axis=None)


In [13]:
GM = 1
x = np.array([0,0,10,0,  0,0,0,20])#Initial positions and velocities

print(two_body_ODE(0,x,GM))


[ 0.e+00  0.e+00  0.e+00  2.e+01  1.e-02  0.e+00 -1.e-02 -0.e+00]


# Burrau's problem

In [14]:
# ODE function
def three_body_ODE(t,y, G, M):
  #Dividing y in positions and velocities
  r1, r2, r3 = y[0:2], y[2:4], y[4:6]  
  v1, v2, v3 = y[6:8], y[8:10], y[10:12] 
  #Calculating the accelerations
  r_12 = r2-r1
  r_13 = r3-r1
  r_23 = r3-r2
  
  a_12 = G*M[0]*M[1]*r_12/np.linalg.norm(r_12)**3
  a_13 = G*M[0]*M[2]*r_13/np.linalg.norm(r_13)**3
  a_23 = G*M[1]*M[2]*r_23/np.linalg.norm(r_23)**3
  #With concatenate I concatenate vectors side by side to get the state vector

  return np.concatenate((v1, v2, v3,  a_12+a_13, -a_12+a_23, -a_13-a_23), axis=None)

def h(h0, y, N=1):
  r1, r2, r3 = y[0:2], y[2:4], y[4:6]  
  #Calculating the accelerations
  r_12 = r2-r1
  r_13 = r3-r1
  r_23 = r3-r2

  return h0/(r_12**-2.0+r_13**-2.0+r_23**-2)**N



In [15]:
G = 1
M = [3,4,5]
#initial positions
y = np.array([1,3,-2,-1,1,-1, 0,0,0,0,0,0])


print(three_body_ODE(1,y, G, M))


[ 0.          0.          0.          0.          0.          0.
 -0.288      -1.3215      2.51022222  0.384      -2.22222222  0.9375    ]
